In [1]:
# from fastbook import *
from sklearn import datasets
import seaborn as sns
import sklearn
import torch

from mean_classifier import MeanClassifier
from gradient_descent_regressor import GradientDescentRegressor
import utils

# Mean Classifier

In [4]:
X_train, X_test, y_train, y_test = utils.load_mnist()

In [5]:
clf = MeanClassifier(loss='mae')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9514229893684387

In [6]:
clf = MeanClassifier(loss='mse')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9661432504653931

# Gradient Descent

In [47]:
M_TRUE = -2
C_TRUE = 0

X = torch.tensor(range(50)).reshape(50, 1).float()
y = ((X * M_TRUE) + C_TRUE).reshape(50)

g = GradientDescentRegressor(epochs=10_000)
g.fit(X, y)
print(g.weights, g.bias)

tensor([-1.9132], requires_grad=True) tensor([0.0026], requires_grad=True)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d3089e79-0e42-40a4-9bfd-5171fd4144aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>